## Analysis
A notebook for plotting the results

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
exp_df = pd.read_json("results/H2_fullrun.json")
exp_df.columns

In [ ]:
for c, i in exp_df.iterrows():
    print(f"{c}: {i['optimizer']}, {i['ansatz_name']}, {i['transformation']}")
    #print("\t:", vqe_res['n_iterations'])
    #print(vqe_res)
    print("\t", i["vqe_output"]["converged_energy"], i["vqe_output"]["n_params"])
    #print(vqe_res['n_iterations'])


In [ ]:
def depth_nice(x):
    x = str(x)
    if x[-2:] == ".0":
        x = x[:-2]
    return x

In [ ]:
ansatz_full_names = np.array([f'{i[1]["ansatz_name"]} ({depth_nice(i[1]["ansatz_depth"])})' for i in exp_df.iterrows()])
u_ansatz_full_names = np.unique(ansatz_full_names)
u_ansatz_full_names

In [ ]:
optims = np.unique(exp_df["optimizer"])
print(optims)

# group according to gradient order
optims = ['Nelder-Mead', 'COBYLA', 'POWELL', 'CG', 'BFGS', 'L-BFGS-B']

In [ ]:
%matplotlib notebook

fig, axs = plt.subplots(nrows=2, figsize=(10,4), sharex="col")

for c, i in enumerate(u_ansatz_full_names):
    exp_df_sub = exp_df[ansatz_full_names == i]
    
    x_jw = []
    x_bk = []
    
    for j in optims:
        exp_df_sub2 = exp_df_sub[exp_df_sub["optimizer"] == j]
        #print(exp_df_sub2["transformation"])
        x_jw.append(exp_df_sub2.iloc[0]["vqe_output"]["converged_energy"] - exp_df.iloc[0]["fci_energy"] + 1e-14)
        x_bk.append(exp_df_sub2.iloc[1]["vqe_output"]["converged_energy"] - exp_df.iloc[0]["fci_energy"] + 1e-14)
    
    axs[0].bar(np.arange(len(x_jw))+c*0.1, x_jw, width=0.1, label=i)
    axs[1].bar(np.arange(len(x_bk))+c*0.1, x_bk, width=0.1, label=i)


for i in [0, 1]:
    axs[i].axhline(y=exp_df.iloc[0]["hf_energy"]-exp_df.iloc[0]["fci_energy"], c='k', label="HF", ls='--')
    axs[i].set_yscale("log")


axs[0].set_ylabel("VQE Error (H) - JW", fontsize=10)
axs[1].set_ylabel("VQE Error (H) - BK", fontsize=10)


axs[1].set_xticks(np.arange(len(x_jw))+0.2)
axs[1].set_xticklabels(optims, rotation='horizontal', fontsize=10)


axs[0].legend(fontsize=6)

#fig.tight_layout()
fig.savefig("out.pdf")